# Kinetic Modeling Workflow

If this notebook is run in Colab, the runtime might need to be restarted after the first execution of the following setup cell.

In [1]:
# Notebook setup
import os
import sys
import glob

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
        # Clone the repository
    !git clone https://github.com/EnzymeML/slac_modeling.git
    %cd slac_modeling

    # Install the requirements
    !pip install -q -r requirements.txt


from MTPHandler.core import Plate
from MTPHandler.readers import read_spectramax
from EnzymePynetics.core import Estimator

## Oxidation of ABTS by SLAC

In the following, the oxidation of 2,2'-azino-bis(3-ethylbenzothiazoline-6-sulfonic acid) (ABTS) by the [small laccase](https://www.uniprot.org/uniprotkb/Q9XAL8/entry/Q9P7I0) (SLAC) in the temperature range from 25 - 45°C is investigated.  
ABTS + O<sub>2</sub> &rarr; ABTS<sup>&middot;+</sup> + H<sub>2</sub>O  


The reaction was monitored the absorbance at 340 nm for 15 min.
The assay way carried out in 96-well plates, whereas enzyme reactions were prepared in triplicates in the ABTS concentration range of 0 - 200 µM in rows A-C. Additionally, a calibration standard was prepared in the same ABTS concentration range in rows D-F.


## Read in plate reader data

Each assay at a given temperature was measured on one plate, whereas enzyme reactions and calibration standard for ABTS concentration calculation are on the same plate. Thus, the five `TXT` outputs of the SpectraMax i3 Multimode Reader (Molecular Devices) were subjected by the workflow by iterating over the files in the `data/raw_data` folder.  
Missing information, essential for the workflow, but lacking in the output files is provided as arguments to the reader method of the `Plate`. In this case information on pH value and the time unit of the measurement is lacking.  
Due to the modularity of the workflow, additional reader methods can be implemented for other plate readers or other file formats.

In [2]:
plates = []
for path in sorted(glob.glob("data/raw_data/*")):
    plates.append(
        Plate.from_reader(read_spectramax, path=path, ph=3.0, time_unit="min")
    )

The measurement data of a `Plate` can be visualized by calling the `visualize()` method.

In [3]:
plates[-1].visualize(wavelengths=340)

### Define species

After loading the data, the species present in the assay are defined. In this case, the species are ABTS and the oxidized form of ABTS, the buffer, and SLAC are defined. Additionally, species which are not explicitly measured, but are part of the reaction to be described are defined.  
In this process, the name, the initial concentration of the species, the corresponding unit added by calling the `add_reactant()` or `add_protein()` method of the `Plate`. Furthermore, SMILES and InChI code of the reactants or UniProtID, sequence, and EC-Number are specified. 

Initial concentrations of different species can be assigned to individual `Well` objects by calling `assign_species()`. The function maps arrays of initial concentrations `to='rows'` or `to='columns'`. Alternatively, a single concentration can be mapped `to='all'` or wells except for specific wells (`to='except'`). 

In [4]:
for plate in plates:
    # Assign buffer to all wells
    buffer = plate.add_reactant(
        id="s0",
        name="sodium acetate",
        constant=True,
        init_conc=50,
        unit="mM",
    )
    plate.assign_species(species=buffer, to="all", init_conc=50, conc_unit="mmol / l")

    # Define ABTS
    abts = plate.add_reactant(
        id="s1",
        name="ABTS",
        constant=False,
        inchi="InChI=1S/C18H18N4O6S4/c1-3-21-13-7-5-11(31(23,24)25)9-15(13)29-17(21)19-20-18-22(4-2)14-8-6-12(32(26,27)28)10-16(14)30-18/h5-10H,3-4H2,1-2H3,(H,23,24,25)(H,26,27,28)/b19-17-,20-18+",
        smiles=r"CCn\1c2ccc(cc2s/c1=N/N=c\3/n(c4ccc(cc4s3)S(=O)(=O)O)CC)S(=O)(=O)O",
    )

    # Assign initial ABTS concentrations row-wise
    plate.assign_species(
        species=abts,
        to="rows",
        init_conc=[0, 5, 10, 15, 25, 50, 75, 100, 150, 200],
        conc_unit="umol / l",
        ids=["A", "B", "C", "D", "E", "F"],
    )

    # Define SLAC
    slac = plate.add_protein(
        id="p0",
        name="Small laccase (SLAC)",
        constant=False,
        ecnumber="1.10.3.2",
        uniprotid="Q9XAL8",
        sequence="MDRRGFNRRVLLGGAAAATSLSIAPEVAGAAPAAKGITARTAPAGGEVRHLKMYAEKLADGQMGYGFEKGKASVPGPLIEVNEGDTLHIEFTNTMDVRASLHVHGLDYEISSDGTAMNKSDVEPGGTRTYTWRTHKPGRRDDGTWRPGSAGYWHYHDHVVGTEHGTGGIRNGLYGPVIVRRKGDVLPDATHTIVFNDMTINNRKPHTGPDFEATVGDRVEIVMITHGEYYHTFHMHGHRWADNRTGILTGPDDPSRVIDNKITGPADSFGFQIIAGEGVGAGAWMYHCHVQSHSDMGMVGLFLVKKPDGTIPGYEPHEHGGATAKSGESGEPTGGAAAHEHEH",
    )
    # Assign initial SLAC concentration to all wells in rows A-C
    plate.assign_species(
        species=slac,
        to="rows",
        ids=["A", "B", "C"],
        init_conc=0.073938727,
        conc_unit="umol / l",
    )

    # Add unobserved species
    # Add oxidised ABTS
    abts_oxidised = plate.add_reactant(
        id="s2",
        name="oxidised ABTS radical",
        constant=False,
        inchi="InChI=1S/C18H18N4O6S4/c1-3-21-13-7-5-11(31(23,24)25)9-15(13)29-17(21)19-20-18-22(4-2)14-8-6-12(32(26,27)28)10-16(14)30-18/h5-10H,3-4H2,1-2H3,(H,23,24,25)(H,26,27,28)/q+1/p-2",
        smiles=r"CCN\1C2=C(C=C(C=C2)S(=O)(=O)[O-])S/C1=N\N=C\3/[N+](C4=C(S3)C=C(C=C4)S(=O)(=O)[O-])CCCCn1/c(=N/N=c/2\sc3c(n2CC)ccc(c3)S(=O)(=O)O)/sc4c1ccc(c4)S(=O)(=O)O",
    )
    # Assign oxidised ABTS to all rows containing SLAC (A-C)
    plate.assign_species(
        species=abts_oxidised,
        to="rows",
        ids=["A", "B", "C"],
        init_conc=0,
        conc_unit="umol / l",
    )

    # Define Oxygen species
    oxygen = plate.add_reactant(
        id="s3", name="Oxygen", constant=False, inchi="InChI=1S/O", smiles="O=O"
    )

    # Define Water species
    water = plate.add_reactant(
        id="s4",
        name="Water",
        constant=False,
        inchi="InChI=1S/H2O/h1H2",
        smiles="[H]O[H]",
    )

Assigned sodium acetate to all wells.
Assigned ABTS with concentrations of [0, 5, 10, 15, 25, 50, 75, 100, 150, 200] umol / l to row A.
Assigned ABTS with concentrations of [0, 5, 10, 15, 25, 50, 75, 100, 150, 200] umol / l to row B.
Assigned ABTS with concentrations of [0, 5, 10, 15, 25, 50, 75, 100, 150, 200] umol / l to row C.
Assigned ABTS with concentrations of [0, 5, 10, 15, 25, 50, 75, 100, 150, 200] umol / l to row D.
Assigned ABTS with concentrations of [0, 5, 10, 15, 25, 50, 75, 100, 150, 200] umol / l to row E.
Assigned ABTS with concentrations of [0, 5, 10, 15, 25, 50, 75, 100, 150, 200] umol / l to row F.
Assigned Small laccase (SLAC) with concentrations of [0.073938727, 0.073938727, 0.073938727, 0.073938727, 0.073938727, 0.073938727, 0.073938727, 0.073938727, 0.073938727, 0.073938727] umol / l to row A.
Assigned Small laccase (SLAC) with concentrations of [0.073938727, 0.073938727, 0.073938727, 0.073938727, 0.073938727, 0.073938727, 0.073938727, 0.073938727, 0.073938727, 

### Blank absorption data

In the next step, the contribution of buffer and enzyme is subtracted from the absorption data. Blanking is carried out adaptively, utilizing the defined initial conditions of each `Well`, which were defined previously.  

In [5]:
for plate in plates:
    plate.blank_species(species=buffer, wavelength=340)
    plate.blank_species(species=slac, wavelength=340)

Mean absorption of sodium acetate at 50.0 mmol / l: 0.03316666666666667
Blanked 60 wells containing sodium acetate.
Mean absorption of Small laccase (SLAC) at 0.073938727 umol / l: 0.0010575757575757537
Blanked 30 wells containing Small laccase (SLAC).
Mean absorption of sodium acetate at 50.0 mmol / l: 0.039357575757575765
Blanked 60 wells containing sodium acetate.
Mean absorption of Small laccase (SLAC) at 0.073938727 umol / l: -0.0004606060606060675
Blanked 30 wells containing Small laccase (SLAC).
Mean absorption of sodium acetate at 50.0 mmol / l: 0.039539393939393944
Blanked 60 wells containing sodium acetate.
Mean absorption of Small laccase (SLAC) at 0.073938727 umol / l: 0.00011818181818181399
Blanked 30 wells containing Small laccase (SLAC).
Mean absorption of sodium acetate at 50.0 mmol / l: 0.03548484848484849
Blanked 60 wells containing sodium acetate.
Mean absorption of Small laccase (SLAC) at 0.073938727 umol / l: 0.002133333333333331
Blanked 30 wells containing Small l

### Concentration calculation and creation of `EnzymeMLDocument` objects

In the next cell, the calibration data of each plate is detected and subjected to a `Calibrator`, to compare different calibration equations, describing the relationship between absorption and concentration af ABTS.  
Therefore, the `calibrate()` method is called, and the respective `Species` and the wavelength of the signal to be used for calibration is specified. Furthermore, an upper signal cutoff can be specified, omitting calibration samples with higher signals.

Visualization and comparison of the fit statistics reviled, that the calibration range needs to be limited to 0-150 µM of ABTS, since for concentrations above 150 µM, the absorption is not linear anymore. In order to describe this non-linear relationship with a polynomial, not enough sampling data between 150 µM and 200 µM is available. Thus accurate concentration calculation would not be possible in this range.  
In consequence, an upper absorption limit of $E_{340} = 3.1$ was chosen, which excludes calibration measurements with ABTS concentrations above 150 µM.

In [6]:
enzymemls = []

for plate in plates:
    print(f"--- ABTS calibration at {plate.temperatures[0]}°C ---\n")
    abts_calibration = plate.calibrate(
        species=abts,
        wavelength=340,
        cutoff=3.1,
        signal_type="Absorbance",
    )

    # Get predefined calibration models
    linear, quadratic, cubic = abts_calibration.models

    # Fit models
    abts_calibration.fit_models()
    abts_calibration.visualize()
    abts_standard = abts_calibration.save_model(linear)

    # Serialize calibration results as json
    with open(
        f"data/calibrations/{abts_standard.name}_standard_{abts_standard.temperature}C.json",
        "w",
    ) as f:
        f.write(abts_standard.json())

    # Map data from a `Plate` to an `EnzymeMLDocument`
    enzymemls.append(
        plate.to_enzymeml(
            name=f"SLAC kinetic assay",
            detected_reactant=abts,
            wavelength=340,
            reactant_standard=abts_standard,
        )
    )

--- ABTS calibration at 25.0°C ---



,AIC,R squared,RMSD
Model Name,,,
linear,-157,0.9965,0.0525
quadratic,-155,0.9965,0.0525
cubic,-153,0.9966,0.0523


Found 27 catalyzed wells
--- ABTS calibration at 30.0°C ---



,AIC,R squared,RMSD
Model Name,,,
cubic,-246,0.9999,0.0094
quadratic,-238,0.9998,0.0113
linear,-235,0.9998,0.0124


Found 27 catalyzed wells
--- ABTS calibration at 35.5°C ---



...

Found 27 catalyzed wells
--- ABTS calibration at 40.0°C ---



,AIC,R squared,RMSD
Model Name,,,
cubic,-250,0.9999,0.0087
linear,-222,0.9997,0.0159
quadratic,-220,0.9997,0.0158


Found 27 catalyzed wells
--- ABTS calibration at 45.0°C ---



,AIC,R squared,RMSD
Model Name,,,
linear,-170,0.9978,0.0415
quadratic,-168,0.9978,0.0414
cubic,-166,0.9978,0.0412


Found 27 catalyzed wells


Since calibration data only allows to calculate the concentration of ABTS up to 150 µM, measurement entries with an initial concentration of 200 µM ABTS are removed from the dataset.

In [7]:
# Remove measurement entries with initial ABTS concentration of 200 µM
for enzymeml in enzymemls:
    del enzymeml.measurements[-1]

## Parameter estimation on the full time-course data

An EnzymePynetics `Estimator` is initialized for each `EnzymeMLDocument`, which was create from individual `Plate` objects.
Next up, the complete oxidation reaction is formulated, creating a `Reaction`, which is added to the `Estimator` using the `add_reaction()` method. Thereby, the previously defined reactants are defined as educts or products.  

In [8]:
estimators = []
for enzymeml in enzymemls:
    estimator, _ = Estimator.from_enzymeml(enzymeml, abts)

    estimator.add_reaction(
        id="r0",
        name="Oxidation",
        educt=[abts, oxygen],
        product=[abts_oxidised, water],
        catalyst=slac,
    )

    estimators.append(estimator)

In the next cell, possible rate laws for the substrate and enzyme concentrations are defined. Possible parameters are the turnover number $k_{cat}$, Michaelis cosntant $K_M$, compeitive inhibition constant $K_{ic}$, uncompetitive inhibition constant $K_{iu}$, and enzyme inactivation rate $k_{ie}$.

In [9]:
for estimator in estimators:
    # Substrate rate laws
    michaelis = estimator.add_model(
        id="model1",
        name="michaelis menten",
        equation="substrate = -(substrate * catalyst * k_cat) / (K_M + substrate)",
    )

    competitive_product = estimator.add_model(
        id="model2",
        name="competitive product inhibition",
        equation="substrate = -(substrate * catalyst * k_cat) / (K_M * (1 + product / K_ic) + substrate)",
    )

    substrate_inhibition = estimator.add_model(
        id="model3",
        name="substrate inhibition",
        equation="substrate = -(k_cat * catalyst * substrate) / (K_M + ((1+(substrate/K_iu))*substrate))",
    )

    # Enzyme rate laws
    enzyme_inactivation = estimator.add_model(
        id="model4",
        name="enzyme inactivation",
        equation="catalyst = -k_ie * catalyst",
    )

In the following step, model creation and subsequent fitting is carried out. Based on the defined rate laws, substrate and enzyme rate laws combined in the background to `ReactionSystem` objects, representing a s system of differential equations. Additionally, each defined substrate rate law is fitted, assuming constant catalytically active enzyme concentration over time.  
Fitting is preformed by calling the `fit_models()` method of the `Estimator`. Furthermore, the inter-parameter correlations can be visualized with `correlations()`.

In [10]:
for estimator in estimators:
    print(f"--- {estimator.name} at {estimator.temperature}°C ---\n")
    estimator.fit_models()
    estimator.correlations()

--- SLAC kinetic assay at 25.0°C ---



  0%|          | 0/6 [00:00<?, ?it/s]

Fitting substrate inhibition with enzyme inactivation model: 100%|██████████| 6/6 [00:07<00:00,  1.26s/it]          


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,umol / l,1 / min / umol / l,umol / l,umol / l,1 / min
Model,,,,,,,
competitive product inhibition with enzyme inactivation,-669,28.775 ± 1 %,25.128 ± 3 %,1.145 ± 4 %,9.858 ± 12 %,,0.056 ± 5 %
michaelis menten with enzyme inactivation,-554,28.132 ± 1 %,31.459 ± 2 %,0.894 ± 3 %,,,0.075 ± 4 %
substrate inhibition with enzyme inactivation,-452,38.865 ± 5 %,52.254 ± 7 %,0.744 ± 9 %,,375.000 ± 15 %,0.076 ± 4 %
competitive product inhibition,-425,26.441 ± 3 %,15.223 ± 10 %,1.737 ± 11 %,2.009 ± 15 %,,
michaelis menten,-147,18.853 ± 2 %,31.588 ± 5 %,0.597 ± 5 %,,,
substrate inhibition,-117,25.986 ± 4 %,52.623 ± 7 %,0.494 ± 8 %,,375.000 ± 9 %,


--- SLAC kinetic assay at 30.0°C ---



...

Fitting substrate inhibition with enzyme inactivation model: 100%|██████████| 6/6 [00:07<00:00,  1.24s/it]          


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,umol / l,1 / min / umol / l,umol / l,umol / l,1 / min
Model,,,,,,,
competitive product inhibition with enzyme inactivation,-284,54.309 ± 4 %,40.547 ± 5 %,1.339 ± 7 %,10.062 ± 15 %,,0.044 ± 10 %
competitive product inhibition,-212,74.629 ± 9 %,44.999 ± 10 %,1.658 ± 14 %,3.573 ± 13 %,,
michaelis menten with enzyme inactivation,-176,41.881 ± 2 %,40.628 ± 4 %,1.031 ± 5 %,,,0.075 ± 6 %
substrate inhibition with enzyme inactivation,-90,58.792 ± 10 %,68.247 ± 13 %,0.861 ± 16 %,,375.000 ± 34 %,0.075 ± 6 %
michaelis menten,63,28.269 ± 2 %,40.996 ± 6 %,0.690 ± 6 %,,,
substrate inhibition,103,39.671 ± 14 %,69.022 ± 20 %,0.575 ± 24 %,,375.000 ± 53 %,


--- SLAC kinetic assay at 45.0°C ---



Fitting substrate inhibition with enzyme inactivation model: 100%|██████████| 6/6 [00:07<00:00,  1.24s/it]          


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,umol / l,1 / min / umol / l,umol / l,umol / l,1 / min
Model,,,,,,,
competitive product inhibition with enzyme inactivation,-228,50.701 ± 2 %,40.590 ± 4 %,1.249 ± 4 %,28.009 ± 30 %,,0.058 ± 9 %
michaelis menten with enzyme inactivation,-215,50.045 ± 2 %,43.144 ± 3 %,1.160 ± 4 %,,,0.075 ± 4 %
competitive product inhibition,-150,52.027 ± 4 %,32.596 ± 8 %,1.596 ± 9 %,4.300 ± 11 %,,
substrate inhibition with enzyme inactivation,-90,71.253 ± 8 %,73.855 ± 11 %,0.965 ± 14 %,,375.000 ± 23 %,0.075 ± 5 %
michaelis menten,109,33.865 ± 2 %,43.647 ± 6 %,0.776 ± 6 %,,,
substrate inhibition,155,48.222 ± 13 %,74.961 ± 19 %,0.643 ± 23 %,,375.000 ± 36 %,


Besides the fit statistics, visualization of the fit provides an additional measure for fit quality assessment. An interactive visualization of the fit together with the measured data is generated by calling the `visualize()` method.

In [11]:
for estimator in estimators:
    estimator.visualize()

As a result of the parameter estimation on the full data set, substrate both substrate inhibition models failed to explain the observed time-course behavior. Therefore, these models are are not further examined as model candidates for the next fitting approach.

## Partially fixed-value parameter estimation

As a second fitting approach, $k_{cat}$ and $K_M$ are estimated based on the data of the first three minutes only, assuming product inhibition and enzyme inactivation are negligible. In a subsequent fitting step, all parameters besides $k_{cat}$ and $K_M$ are optimized based on the full data set.

As for the previous fitting approach, rate laws are defined, after new `Estimator` objects are initialized based on the `EnzymeMLDocument` objects.

This time, the `max_time` argument of the `fit_models()` method is used to limit the used data for fitting to the first three minutes.

In [12]:
init_rate_estimators = []
for enzymeml in enzymemls:
    estimator, _ = Estimator.from_enzymeml(enzymeml, abts)

    print(f"--- {estimator.name} at {estimator.temperature}°C ---\n")

    estimator.add_reaction(
        id="r0", name="Oxidation", educt=abts, product=abts_oxidised, catalyst=slac
    )

    estimator.add_model(
        id="model1",
        name="michaelis menten",
        equation="substrate = -(substrate * catalyst * k_cat) / (K_M + substrate)",
    )

    estimator.add_model(
        id="model2",
        name="competitive product inhibition",
        equation="substrate = -(substrate * catalyst * k_cat) / (K_M * (1 + product / K_ic) + substrate)",
    )

    # Enzyme rate laws
    estimator.add_model(
        id="model4",
        name="enzyme inactivation",
        equation="catalyst = -k_ie * catalyst",
    )

    estimator.fit_models(max_time=3)

    init_rate_estimators.append(estimator)

--- SLAC kinetic assay at 25.0°C ---



Fitting competitive product inhibition with enzyme inactivation model: 100%|██████████| 4/4 [00:03<00:00,  1.29it/s]


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,umol / l,1 / min / umol / l,umol / l,umol / l,1 / min
Model,,,,,,,
competitive product inhibition,-203,26.091 ± 4 %,15.312 ± 18 %,1.704 ± 18 %,3.712 ± 34 %,,
competitive product inhibition with enzyme inactivation,-201,27.111 ± 7 %,15.519 ± 18 %,1.747 ± 19 %,3.879 ± 36 %,,0.032 ± > 100 %
michaelis menten with enzyme inactivation,-185,26.356 ± 8 %,22.914 ± 11 %,1.150 ± 13 %,,,0.079 ± 72 %
michaelis menten,-185,23.811 ± 3 %,22.940 ± 11 %,1.038 ± 11 %,,,


--- SLAC kinetic assay at 30.0°C ---



Fitting competitive product inhibition with enzyme inactivation model: 100%|██████████| 4/4 [00:03<00:00,  1.16it/s]


...

Fitting competitive product inhibition with enzyme inactivation model: 100%|██████████| 4/4 [00:03<00:00,  1.03it/s]


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,umol / l,1 / min / umol / l,umol / l,umol / l,1 / min
Model,,,,,,,
competitive product inhibition with enzyme inactivation,-158,56.880 ± 11 %,34.737 ± 12 %,1.637 ± 16 %,10.822 ± 42 %,,0.118 ± 45 %
competitive product inhibition,-155,53.226 ± 12 %,35.443 ± 13 %,1.502 ± 18 %,8.052 ± 37 %,,
michaelis menten with enzyme inactivation,-150,46.372 ± 7 %,33.259 ± 10 %,1.394 ± 12 %,,,0.163 ± 33 %
michaelis menten,-142,37.821 ± 4 %,33.342 ± 11 %,1.134 ± 11 %,,,


--- SLAC kinetic assay at 45.0°C ---



Fitting competitive product inhibition with enzyme inactivation model: 100%|██████████| 4/4 [00:03<00:00,  1.00it/s]


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,umol / l,1 / min / umol / l,umol / l,umol / l,1 / min
Model,,,,,,,
competitive product inhibition with enzyme inactivation,-175,57.727 ± 5 %,36.261 ± 8 %,1.592 ± 10 %,27.017 ± 69 %,,0.141 ± 29 %
michaelis menten with enzyme inactivation,-175,57.534 ± 5 %,37.980 ± 7 %,1.515 ± 9 %,,,0.165 ± 23 %
competitive product inhibition,-166,50.108 ± 4 %,34.886 ± 10 %,1.436 ± 10 %,13.100 ± 39 %,,
michaelis menten,-158,46.898 ± 3 %,38.111 ± 8 %,1.231 ± 9 %,,,


### Visualize $\frac{k_{cat}}{K_M}$ over temperature for the different estimation approaches

In [13]:
import plotly.graph_objects as go

model_name = "michaelis menten with enzyme inactivation"

fig = go.Figure()

# Add traces for full time-course parameter estimation
for estimator_id, estimator in enumerate(estimators):
    system = [
        system for system in estimator.reaction_systems if system.name == model_name
    ][0]
    # Extract parameters
    for reaction in system.reactions:
        if reaction.model.equation.startswith("substrate"):
            for parameter in reaction.model.parameters:
                if parameter.name == "k_cat":
                    k_cat = parameter.value
                    k_cat_stderr = parameter.stdev
                    k_cat_unit = parameter.unit
                if parameter.name == "K_M":
                    K_m = parameter.value
                    K_m_stderr = parameter.stdev
                    K_m_unit = parameter.unit
            kcat_km_corr = system.get_correlation(
                param_1="k_cat", param_2="K_M")
            kcat_km = k_cat / K_m
            stderr = ((k_cat_stderr / k_cat) ** 2 +
                      (K_m_stderr / K_m) ** 2) ** 0.5
            if estimator_id == 0:
                showlegend = True
            else:
                showlegend = False

            # Add traces for full time-course parameters
            fig.add_trace(
                go.Scatter(
                    y=[kcat_km],
                    x=[reaction.temperature],
                    name=f"full time-course<br>parameter estimation",
                    legendgroup="full time-course",
                    mode="markers",
                    marker=dict(
                        size=10,
                        cmax=45,
                        cmin=25,
                        color="blue",
                        colorscale="bluered",
                    ),
                    error_y=dict(
                        type="data",
                        array=[stderr],
                        arrayminus=[stderr],
                        visible=True,
                        thickness=1,
                        color="black",
                    ),
                    showlegend=showlegend,
                )
            )

# add traces for partially fixed-value parameter estimation
for estimator_id, estimator in enumerate(init_rate_estimators):
    system = [
        system for system in estimator.reaction_systems if system.name == model_name
    ][0]
    for reaction in system.reactions:
        if reaction.model.equation.startswith("substrate"):
            for parameter in reaction.model.parameters:
                if parameter.name == "k_cat":
                    k_cat = parameter.value
                    k_cat_stderr = parameter.stdev
                    k_cat_unit = parameter.unit
                if parameter.name == "K_M":
                    K_m = parameter.value
                    K_m_stderr = parameter.stdev
                    K_m_unit = parameter.unit
            kcat_km_corr = system.get_correlation(
                param_1="k_cat", param_2="K_M")

            kcat_km = k_cat / K_m
            stderr = ((k_cat_stderr / k_cat) ** 2 +
                      (K_m_stderr / K_m) ** 2) ** 0.5
            if estimator_id == 0:
                showlegend = True
            else:
                showlegend = False
            fig.add_trace(
                go.Scatter(
                    y=[kcat_km],
                    x=[reaction.temperature],
                    name=f"partially fixed-value<br>parameter estimation",
                    legendgroup="Initial rates",
                    mode="markers",
                    marker_symbol="triangle-up",
                    marker=dict(
                        size=10,
                        cmax=45,
                        cmin=25,
                        color="orange",
                        colorscale="bluered",
                    ),
                    showlegend=showlegend,
                    error_y=dict(
                        type="data",
                        array=[stderr],
                        arrayminus=[stderr],
                        visible=True,
                        thickness=1,
                        color="black",
                    ),
                )
            )


fig.update_layout(
    template="simple_white",
    yaxis_title=f"k<sub>cat</sub> K<sub>M</sub><sup>-1</sup> / min<sup>-1</sup> µM<sup>-1</sup>",
    xaxis_title=f"Temperature / {reaction.temperature_unit}",
)


config = {
    "toImageButtonOptions": {
        "format": "png",
        "filename": "custom_image",
        "scale": 8,
    }
}


fig.show(config=config)

In the following step, partially fixed-value parameter estimation is performed. Thereby, values for $k_{cat}$ and $K_M$ from the 'michaelis menten' model of the previous fitting steps are fixed, and the remaining parameters are estimated based on the full data set.

In [14]:
for estimator in init_rate_estimators:
    print(f"--- {estimator.name} at {estimator.temperature}°C ---\n")

    estimator.fit_models_fixed_params(
        fixed_params=["k_cat", "K_M"], model="michaelis menten"
    )

--- SLAC kinetic assay at 25.0°C ---



Fitting michaelis menten model: 100%|██████████| 4/4 [00:02<00:00,  1.71it/s]                                       


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,umol / l,1 / min / umol / l,umol / l,umol / l,1 / min
Model,,,,,,,
competitive product inhibition with enzyme inactivation,-514,23.811 ± 8 %,22.940 ± 12 %,1.038 ± 15 %,12.891 ± 11 %,,0.035 ± 8 %
michaelis menten with enzyme inactivation,-398,23.811 ± 9 %,22.940 ± 11 %,1.038 ± 14 %,,,0.061 ± 1 %
competitive product inhibition,-386,23.811 ± 4 %,22.940 ± 12 %,1.038 ± 13 %,4.891 ± 2 %,,
michaelis menten,-185,,,,,,


--- SLAC kinetic assay at 30.0°C ---



Fitting michaelis menten model: 100%|██████████| 4/4 [00:02<00:00,  1.54it/s]                                       


...

Fitting michaelis menten model: 100%|██████████| 4/4 [00:01<00:00,  2.11it/s]                                       


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,umol / l,1 / min / umol / l,umol / l,umol / l,1 / min
Model,,,,,,,
competitive product inhibition with enzyme inactivation,-192,37.821 ± 17 %,33.342 ± 13 %,1.134 ± 21 %,30.396 ± 19 %,,0.041 ± 12 %
michaelis menten with enzyme inactivation,-153,37.821 ± 9 %,33.342 ± 10 %,1.134 ± 14 %,,,0.068 ± 2 %
michaelis menten,-142,,,,,,
competitive product inhibition,-121,37.821 ± 17 %,33.342 ± 14 %,1.134 ± 22 %,10.472 ± 2 %,,


--- SLAC kinetic assay at 45.0°C ---



Fitting michaelis menten model: 100%|██████████| 4/4 [00:01<00:00,  2.21it/s]                                       


,AIC,k_cat,K_M,k_cat / K_M,K_ic,K_iu,k_ie
,,1 / min,umol / l,1 / min / umol / l,umol / l,umol / l,1 / min
Model,,,,,,,
competitive product inhibition with enzyme inactivation,-218,46.898 ± 7 %,38.111 ± 8 %,1.231 ± 10 %,34.010 ± 28 %,,0.052 ± 9 %
michaelis menten with enzyme inactivation,-202,46.898 ± 6 %,38.111 ± 7 %,1.231 ± 10 %,,,0.070 ± 1 %
michaelis menten,-158,,,,,,
competitive product inhibition,-132,46.898 ± 4 %,38.111 ± 9 %,1.231 ± 10 %,7.341 ± 2 %,,


In [15]:
for estimator in init_rate_estimators:
    estimator.visualize()

### Visualize $k_{cat}$ over temperature for the different estimation approaches

In [16]:
model_name = "michaelis menten with enzyme inactivation"

fig = go.Figure()

# Add traces for full time-course parameter estimation
for estimator_id, estimator in enumerate(estimators):
    system = [
        system for system in estimator.reaction_systems if system.name == model_name
    ][0]
    for reaction in system.reactions:
        if reaction.model.equation.startswith("substrate"):
            for parameter in reaction.model.parameters:
                if parameter.name == "k_cat":
                    k_cat = parameter.value
                    k_cat_stderr = parameter.stdev
                    k_cat_unit = parameter.unit

            if estimator_id == 0:
                showlegend = True
            else:
                showlegend = False
            fig.add_trace(
                go.Scatter(
                    y=[k_cat],
                    x=[reaction.temperature],
                    name=f"full time-course<br>parameter estimation",
                    legendgroup="full timecourse",
                    mode="markers",
                    marker=dict(
                        size=10,
                        cmax=45,
                        cmin=25,
                        color="blue",
                        colorscale="bluered",
                    ),
                    error_y=dict(
                        type="data",
                        array=[k_cat_stderr],
                        arrayminus=[k_cat_stderr],
                        visible=True,
                        thickness=1,
                        color="black",
                    ),
                    showlegend=showlegend,
                )
            )

# add traces for partially fixed-value parameter estimation
for estimator_id, estimator in enumerate(init_rate_estimators):
    system = [
        system for system in estimator.reaction_systems if system.name == model_name
    ][0]
    for reaction in system.reactions:
        if reaction.model.equation.startswith("substrate"):
            for parameter in reaction.model.parameters:
                if parameter.name == "k_cat":
                    k_cat = parameter.value
                    k_cat_stderr = parameter.stdev
                    k_cat_unit = parameter.unit

            if estimator_id == 0:
                showlegend = True
            else:
                showlegend = False
            fig.add_trace(
                go.Scatter(
                    y=[k_cat],
                    x=[reaction.temperature],
                    name=f"partially fixed-value<br>parameter estimation",
                    legendgroup="Initial rates",
                    mode="markers",
                    marker_symbol="triangle-up",
                    marker=dict(
                        size=10,
                        cmax=45,
                        cmin=25,
                        color="orange",
                        colorscale="bluered",
                    ),
                    showlegend=showlegend,
                    error_y=dict(
                        type="data",
                        array=[k_cat_stderr],
                        arrayminus=[k_cat_stderr],
                        visible=True,
                        thickness=1,
                        color="black",
                    ),
                )
            )


fig.update_layout(
    template="simple_white",
    yaxis_title=f"k<sub>cat</sub> / min<sup>-1</sup>",
    xaxis_title=f"Temperature / {reaction.temperature_unit}",
)

config = {
    "toImageButtonOptions": {
        "format": "png",
        "filename": "custom_image",
        "scale": 8,
    }
}


fig.show(config=config)

### Visualize Michaelis constant for ABTS over temperature for the different estimation approaches

In [17]:
fig = go.Figure()


# Add traces for full time-course parameter estimation
for estimator_id, estimator in enumerate(estimators):
    model_name = "michaelis menten with enzyme inactivation"
    system = [
        system for system in estimator.reaction_systems if system.name == model_name
    ][0]
    for reaction in system.reactions:
        if reaction.model.equation.startswith("substrate"):
            for parameter in reaction.model.parameters:
                if parameter.name == "k_cat":
                    k_cat = parameter.value
                    k_cat_stderr = parameter.stdev
                    k_cat_unit = parameter.unit
                if parameter.name == "K_M":
                    K_m = parameter.value
                    K_m_stderr = parameter.stdev
                    K_m_unit = parameter.unit

            if estimator_id == 0:
                showlegend = True
            else:
                showlegend = False
            fig.add_trace(
                go.Scatter(
                    y=[K_m],
                    x=[reaction.temperature],
                    name=f"full time-course<br>parameter estimation",
                    legendgroup="full timecourse",
                    mode="markers",
                    marker=dict(
                        size=10,
                        cmax=45,
                        cmin=25,
                        color="blue",
                        colorscale="bluered",
                    ),
                    error_y=dict(
                        type="data",
                        array=[K_m_stderr],
                        arrayminus=[K_m_stderr],
                        visible=True,
                        thickness=1,
                        color="black",
                    ),
                    showlegend=showlegend,
                )
            )

# add traces for partially fixed-value parameter estimation
for estimator_id, estimator in enumerate(init_rate_estimators):
    system = [
        system for system in estimator.reaction_systems if system.name == model_name
    ][0]
    for reaction in system.reactions:
        if reaction.model.equation.startswith("substrate"):
            for parameter in reaction.model.parameters:
                if parameter.name == "K_M":
                    K_m = parameter.value
                    K_m_stderr = parameter.stdev
                    K_m_unit = parameter.unit

            if estimator_id == 0:
                showlegend = True
            else:
                showlegend = False
            fig.add_trace(
                go.Scatter(
                    y=[K_m],
                    x=[reaction.temperature],
                    name=f"partially fixed-value<br>parameter estimation",
                    legendgroup="Initial rates",
                    mode="markers",
                    marker_symbol="triangle-up",
                    marker=dict(
                        size=10,
                        cmax=45,
                        cmin=25,
                        color="orange",
                        colorscale="bluered",
                    ),
                    showlegend=showlegend,
                    error_y=dict(
                        type="data",
                        array=[K_m_stderr],
                        arrayminus=[K_m_stderr],
                        visible=True,
                        thickness=1,
                        color="black",
                    ),
                )
            )


fig.update_layout(
    template="simple_white",
    yaxis_title=f"K<sub>M</sub> / µM",
    xaxis_title=f"Temperature / {reaction.temperature_unit}",
)

config = {
    "toImageButtonOptions": {
        "format": "png",
        "filename": "custom_image",
        "scale": 8,
    }
}


fig.show(config=config)

### Visualize competitive inhibition constant over temperature for the different estimation approaches

The temperature dependency of the competitive inhibition constant over temperature is shown in the following figure. Therefore, the competitive inhibition model is used.

In [18]:
fig = go.Figure()


# Add traces for full time-course parameter estimation
for estimator_id, estimator in enumerate(estimators):
    model_name = "competitive product inhibition"
    system = [
        system for system in estimator.reaction_systems if system.name == model_name
    ][0]
    for reaction in system.reactions:
        if reaction.model.equation.startswith("substrate"):
            for parameter in reaction.model.parameters:
                if parameter.name == "K_ic":
                    k_ie = parameter.value
                    k_ie_stderr = parameter.stdev
                    k_ie_unit = parameter.unit
            if estimator_id == 0:
                showlegend = True
            else:
                showlegend = False
            fig.add_trace(
                go.Scatter(
                    y=[k_ie],
                    x=[reaction.temperature],
                    name=f"full time-course<br>parameter estimation",
                    legendgroup="full timecourse",
                    mode="markers",
                    marker=dict(
                        size=10,
                        cmax=45,
                        cmin=25,
                        color="blue",
                        colorscale="bluered",
                    ),
                    showlegend=showlegend,
                    error_y=dict(
                        type="data",  # value of error bar given in data coordinates
                        array=[k_ie_stderr],
                        arrayminus=[k_ie_stderr],
                        visible=True,
                        thickness=1,
                        color="black",
                    ),
                )
            )

# add traces for partially fixed-value parameter estimation
for estimator_id, estimator in enumerate(init_rate_estimators):
    system = [
        system for system in estimator.reaction_systems if system.name == model_name
    ][0]
    for reaction in system.reactions:
        if reaction.model.equation.startswith("substrate"):
            for parameter in reaction.model.parameters:
                if parameter.name == "K_ic":
                    k_ie = parameter.value
                    k_ie_stderr = parameter.stdev
                    k_ie_unit = parameter.unit
            if estimator_id == 0:
                showlegend = True
            else:
                showlegend = False
            fig.add_trace(
                go.Scatter(
                    y=[k_ie],
                    x=[reaction.temperature],
                    name=f"partially fixed-value<br>parameter estimation",
                    legendgroup="partially-fixed parameters",
                    mode="markers",
                    marker_symbol="triangle-up",
                    marker=dict(
                        size=10,
                        cmax=45,
                        cmin=25,
                        color="orange",
                        colorscale="bluered",
                    ),
                    showlegend=showlegend,
                    error_y=dict(
                        type="data",
                        array=[k_ie_stderr],
                        arrayminus=[k_ie_stderr],
                        visible=True,
                        thickness=1,
                        color="black",
                    ),
                )
            )

fig.update_layout(
    template="simple_white",
    yaxis_title=f"K<sub>ic</sub> / µM",
    xaxis_title=f"Temperature / {reaction.temperature_unit}",
)


config = {
    "toImageButtonOptions": {
        "format": "png",
        "filename": "custom_image",
        "scale": 8,
    }
}


fig.show(config=config)

### Visualize enzyme inactivation over temperature for the different estimation approaches


In [19]:
import numpy as np

fig = go.Figure()


# Add traces for full time-course parameter estimation
for estimator_id, estimator in enumerate(estimators):
    model_name = "michaelis menten with enzyme inactivation"
    system = [
        system for system in estimator.reaction_systems if system.name == model_name
    ][0]
    for reaction in system.reactions:
        if reaction.model.equation.startswith("catalyst"):
            for parameter in reaction.model.parameters:
                if parameter.name == "k_ie":
                    k_ie = np.log(2) / parameter.value
                    k_ie_stderr = parameter.stdev
                    k_ie_unit = parameter.unit
            if estimator_id == 0:
                showlegend = True
            else:
                showlegend = False
            fig.add_trace(
                go.Scatter(
                    y=[k_ie],
                    x=[reaction.temperature],
                    name=f"full time-course<br>parameter estimation",
                    legendgroup="full timecourse",
                    mode="markers",
                    marker=dict(
                        size=10,
                        cmax=45,
                        cmin=25,
                        color="blue",
                        colorscale="bluered",
                    ),
                    showlegend=showlegend,
                    error_y=dict(
                        type="data",
                        array=[np.log(2) / (k_ie - k_ie_stderr)],
                        arrayminus=[np.log(2) / (k_ie - k_ie_stderr)],
                        visible=True,
                        thickness=1,
                        color="black",
                    ),
                )
            )

# add traces for partially fixed-value parameter estimation
for estimator_id, estimator in enumerate(init_rate_estimators):
    system = [
        system for system in estimator.reaction_systems if system.name == model_name
    ][0]
    for reaction in system.reactions:
        if reaction.model.equation.startswith("catalyst"):
            for parameter in reaction.model.parameters:
                if parameter.name == "k_ie":
                    k_ie = np.log(2) / parameter.value
                    k_ie_stderr = parameter.stdev
                    k_ie_unit = parameter.unit
            if estimator_id == 0:
                showlegend = True
            else:
                showlegend = False
            fig.add_trace(
                go.Scatter(
                    y=[k_ie],
                    x=[reaction.temperature],
                    name=f"partially fixed-value<br>parameter estimation",
                    legendgroup="partially fixed parameters",
                    mode="markers",
                    marker_symbol="triangle-up",
                    marker=dict(
                        size=10,
                        cmax=45,
                        cmin=25,
                        color="orange",
                        colorscale="bluered",
                    ),
                    showlegend=showlegend,
                    error_y=dict(
                        type="data",
                        array=[np.log(2) / (k_ie - k_ie_stderr)],
                        arrayminus=[np.log(2) / (k_ie - k_ie_stderr)],
                        visible=True,
                        thickness=1,
                        color="black",
                    ),
                )
            )


fig.update_layout(
    template="simple_white",
    yaxis_title=f"half-life / min",
    xaxis_title=f"Temperature / {reaction.temperature_unit}",
)


config = {
    "toImageButtonOptions": {
        "format": "png",
        "filename": "custom_image",
        "scale": 8,
    }
}


fig.show(config=config)

Lastly, the modeling results of the best-fitting `ReactionSystem` are combined with the corresponding `EnzymeMLDocument` objects. Therefore, the `ReactionSystem` is extracted using the name of the model together with the `get_reaction_system()` method. Subsequently, the `EnzymeMLDocument` object is serialized as an `OMEX` archive, using the `to_omex()` method.

In [20]:
# Get the reaction system
result_enzymemls = []
for estimator, enzymeml in zip(init_rate_estimators, enzymemls):
    model_name = "michaelis menten with enzyme inactivation"
    reaction_system = estimator.get_reaction_system(model_name)

    # Save the reaction system to EnzymeML
    result_enzymemls.append(
        estimator.to_omex(
            enzymeml=enzymeml, reaction_system=reaction_system, out_path="data/enzymeml"
        )
    )


Archive was written to data/enzymeml/SLAC_kinetic_assay_25.0C.omex


Archive was written to data/enzymeml/SLAC_kinetic_assay_30.0C.omex


Archive was written to data/enzymeml/SLAC_kinetic_assay_35.5C.omex


Archive was written to data/enzymeml/SLAC_kinetic_assay_40.0C.omex


Archive was written to data/enzymeml/SLAC_kinetic_assay_45.0C.omex

